In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

sys.path.insert(0, "../")

import warnings
warnings.filterwarnings('ignore')

from michael import janet

Getting a `x0 is infeasible` error when running the curve_fit to the wavelet portion. Let's try and identify the target.

In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

In [3]:
index = 2#uni[uni.source_id == int(48977371798954368)].index[0]

In [4]:
for method in ['tess-sip']:#'eleanor', 'eleanor-corner','eleanor-pca', 'unpopular','tess-sip']:
    j = janet.boot(uni, index=index, pipeline = method)
    j.prepare_data()
    j.get_rotation(period_range=(0.2, 40.5))
    j.validate_rotation()
    j.view()
    plt.show()

Target DR3 ID 5311109828992983168 has tesscut data available on MAST for Sectors 8, 9, 36
Data already downloaded for Sector 8.
Data already downloaded for Sector 9.
Data already downloaded for Sector 36.


Running pixels in aperture: 100%|████████████| 300/300 [00:01<00:00, 240.76it/s]


### Running Simple Astropy Lomb-Scargle on Sector 8-9 on star 5311109828992983168 ###
### Completed Simple Astropy Lomb-Scargle for Sector 8-9 on star 5311109828992983168 ###
### Running Wavelet Estimation for Sector 8-9 on star 5311109828992983168 ###


100%|█████████████████████████████████████████| 761/761 [00:20<00:00, 37.73it/s]


ValueError: `x0` is infeasible.

In [9]:
import jazzhands
period_range = j.period_range
sector = '8-9'
# Call the relevant light curve
clc = j.void[f'{j.pl}lc_{sector}']

t = clc.time.value
f = clc.flux.value
wt = jazzhands.WaveletTransformer(t, f)
_, _, wwz, wwa = wt.auto_compute(nu_min = 1./period_range[1], nu_max = 1./period_range[0])

j.void[f'{sector}_wt'] = wt
j.void[f'{sector}_wwz'] = wwz
j.void[f'{sector}_wwa'] = wwa

# Create data to fit
w = np.sum(wwz, axis=1)
w /= w.max() #Normalize
p = 1/wt.nus

max_w = np.max(w)
max_p = p[np.argmax(w)]

s = (p > 0.6*max_p) & (p < 1.4*max_p)
w = w[s]
p = p[s]

# Fit a Gaussian
## Params are mu, sigma, Amplitude
lolim = 0.8*max_p
uplim = 1.2*max_p

# If the max period somehow lies outside the period range, don't adjust
# the period limits
if lolim < period_range[0]:
    lolim = period_range[0]
if uplim > period_range[1]:
    uplim = period_range[1]


100%|█████████████████████████████████████████| 761/761 [00:19<00:00, 38.86it/s]


In [21]:
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
import warnings

import jazzhands

from michael.utils import _gaussian_fn, _safety

In [23]:
popt, pcov = curve_fit(_gaussian_fn, p, w, p0 = [40.5, 0.1*max_p, max_w],
                        bounds = ([lolim, 0., 0.9*max_w],[uplim, 0.25*max_p, 1.1*max_w]))

In [ ]:
raise ValueError

In [ ]:
j.prepare_data()

In [ ]:
any([len(a) > 2 for a in j.sectors])

Cleaning up the data seems to have fixed things. Either way we'll still just change the data downloading process to avoid anything like this in the future.

In [ ]:
sys.exit()

In [ ]:
j.void['r_43-44']['raw_lc'].plot()

In [ ]:
j.void['rawlc_43-44'].plot()

Let's fix the cpm times

In [ ]:
import glob
from astropy.coordinates import SkyCoord
import os
import astropy.units as u
rastr = str(j.ra)
step = len(rastr.split('.')[0])
decstr = str(j.dec)
step = len(decstr.split('.')[0])
sfiles = np.sort(glob.glob(f'{os.path.expanduser("~")}/.eleanor/tesscut/*{rastr[:(6+step)]}*{decstr[:(6+step)]}*'))
coords = SkyCoord(ra = j.ra, dec = j.dec, unit = (u.deg, u.deg))

if len(sfiles) == 0:
    sfiles = np.sort(glob.glob(f'{os.path.expanduser("~")}/.eleanor/tesscut/*{rastr[:(4+step)]}*{decstr[:(4+step)]}*'))
    if len(sfiles) == 0:
        raise ValueError("No tesscut files could be found for this target.")

if len(sfiles) < len(j.sectorlist):
    raise ValueError("There are more sectors available than have been "+
                    "loaded into the sectorlist. Reset the data.")

In [ ]:
sfiles

In [ ]:
from michael.janet import pipelines
f"{', '.join(list(pipelines))}"

In [ ]:
list(j.methods)